In [49]:
import geopandas as gpd
import geojsonio as geoio
import pandas as pd
from collections import defaultdict
from shapely.geometry import Point, Polygon
from collections import Counter

from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

# Read Data

In [50]:
vancouver_all_data = gpd.read_file('VancouverAllData.geojson')
vancouver_gdf = gpd.GeoDataFrame({'population':vancouver_all_data['pop'], 'geometry':vancouver_all_data['geometry']})
vancouver_gdf = vancouver_gdf.reset_index()

osm_data = pd.read_json('osm/amenities-vancouver.json.gz', lines=True, compression='gzip')
osm_data['geometry'] = list(zip(osm_data['lon'],osm_data['lat']))
osm_data['geometry'] = osm_data['geometry'].apply(Point)
osm_gdf = gpd.GeoDataFrame({'amenity':osm_data['amenity'], 'geometry':osm_data['geometry']}, geometry='geometry', crs=vancouver_gdf.crs)

# Data Prep

In [52]:
# use spatial join to match points (osm data) and multipolygons (vancouver data)
pointInPolys = gpd.tools.sjoin(osm_gdf, vancouver_gdf, op="within", how='left')
# index_right is the index of the joined multipolygon, i.e. ID of an area
# group by area and for each area, get the list of amenities located in this area
amenitiesInEachArea = pointInPolys.groupby(['index_right'])['amenity'].apply(list)
# count the number of each amenity in the area and put it into dict
amenitiesInEachArea = amenitiesInEachArea.apply(lambda x: Counter(x))
vancouver_gdf['amenities'] = amenitiesInEachArea
# # split the dict column into separate columns
vancouver_gdf = pd.concat([vancouver_gdf.drop(['amenities'], axis=1), vancouver_gdf['amenities'].apply(pd.Series)], axis=1)
# # some columns are duplicated for some reason??? (removed duplciates based on col names but need to make sure we're not dropping useful data)
vancouver_gdf = vancouver_gdf.loc[:,~vancouver_gdf.columns.duplicated()].drop([0, 'geometry', 'index'], axis=1).fillna(0)

/Users/raufshimarov/opt/anaconda3/envs/geo_env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3301: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


Scale data accroding to the amenities scoring

In [54]:

import json

with open("amenities_scores.txt") as f:
    data = f.read()
scores = json.loads(data)
df_train = vancouver_gdf.drop('population', axis=1)
df_train += 1
scores_vector = []
for i in range(len(df_train.columns)):
    scores_vector.append(scores[df_train.columns[i]])
scores_vector

scaled_df_train = df_train * scores_vector



# Machine Learning

Our x is the number of amenities of some type in the area (e.g. 2 cafes, 4 banks, etc.). Our y is the population density in this area.<br>
TO DO: Did we want to manually calculate a score for each area and use it as training data?

In [56]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler

X = scaled_df_train.values
y = vancouver_gdf['population'].values # divide by the area of the multipolygon 
X_train, X_valid, y_train, y_valid = train_test_split(X, y)
# train model
model = make_pipeline(
    PCA(50),
    PolynomialFeatures(degree=3, include_bias=True),
    LinearRegression(fit_intercept=False)
)
model.fit(X_train, y_train)
# validate model
print(model.score(X_valid, y_valid))

-3.132004093321778e+18


# Heatmap

Red: overpopulated<br>
Yellow: OK<br>
Green: underpopulated<br>

# Analysis